In [1]:
# ---------------------------------------------------------------------------
# Langkah 1: Impor Pustaka dan Muat Data Proyek
# ---------------------------------------------------------------------------
import pandas as pd
import numpy as np
import os

# Path ke file CSV lokal (dari folder model ke subfolder data)
file_path = 'data/data_kpbu.csv'

try:
    # Muat dataset proyek ke dalam DataFrame
    df_proyek = pd.read_csv(file_path)
    print(f"File '{file_path}' berhasil dimuat.")
    print(f"Terdapat {len(df_proyek)} proyek dalam database.")
    
    # Tambahkan kolom Sektor_Proyek berdasarkan id_sektor untuk kompatibilitas
    sektor_mapping = {
        1: 'Air dan Sanitasi', 
        2: 'Infrastruktur', 
        3: 'Jalan dan Jembatan', 
        4: 'Transportasi Laut',
        5: 'Transportasi Udara', 
        6: 'Transportasi Darat', 
        7: 'Telekomunikasi',
        8: 'Transportasi', 
        9: 'Telekomunikasi dan Informatika', 
        10: 'Pengelolaan Sampah',
        11: 'Konservasi Energi', 
        12: 'Sumber Daya Air', 
        13: 'Kesehatan',
        14: 'Penelitian dan Pengembangan', 
        15: 'Perumahan', 
        16: 'Energi Terbarukan'
    }
    
    df_proyek['Sektor_Proyek'] = df_proyek['id_sektor'].map(sektor_mapping)
    
    # Tampilkan info dasar dataset
    print("\n--- Info Dataset Proyek KPBU ---")
    print(f"Kolom yang tersedia: {list(df_proyek.columns)}")
    print("\nDistribusi Profil Risiko:")
    print(df_proyek['Profil_Risiko'].value_counts())
    print("\nDistribusi Sektor:")
    print(df_proyek['Sektor_Proyek'].value_counts())
    
    print("\n--- Contoh Data Proyek ---")
    print(df_proyek[['Nama_Proyek', 'Sektor_Proyek', 'Profil_Risiko', 'Durasi_Konsesi_Tahun', 'Nilai_Investasi_Total_IDR']].head())
    
except FileNotFoundError:
    print(f"File tidak ditemukan: {file_path}")
    print("Pastikan file data_kpbu.csv ada di folder data/")
    raise SystemExit()
except Exception as e:
    print(f"Terjadi kesalahan saat memuat file: {e}")
    raise SystemExit()


File 'data/data_kpbu.csv' berhasil dimuat.
Terdapat 57 proyek dalam database.

--- Info Dataset Proyek KPBU ---
Kolom yang tersedia: ['ID_Proyek', 'Nama_Proyek', 'id_sektor', 'id_status', 'Durasi_Konsesi_Tahun', 'Nilai_Investasi_Total_IDR', 'Profil_Risiko', 'Sektor_Proyek']

Distribusi Profil Risiko:
Profil_Risiko
Tinggi      20
Menengah    19
Rendah      18
Name: count, dtype: int64

Distribusi Sektor:
Sektor_Proyek
Jalan dan Jembatan                19
Air dan Sanitasi                  13
Energi Terbarukan                  5
Telekomunikasi dan Informatika     4
Konservasi Energi                  2
Kesehatan                          2
Transportasi Darat                 2
Penelitian dan Pengembangan        2
Pengelolaan Sampah                 2
Infrastruktur                      1
Transportasi Laut                  1
Sumber Daya Air                    1
Perumahan                          1
Transportasi                       1
Transportasi Udara                 1
Name: count, dtype: int6

In [2]:
# ---------------------------------------------------------------------------
# Langkah 2: Dapatkan Profil Investor Secara Interaktif
# ---------------------------------------------------------------------------

def get_investor_profile_interactive(df):
    """Fungsi untuk mendapatkan profil investor dari input pengguna secara interaktif."""

    print("\n" + "="*60)
    print("           KUESIONER PROFIL INVESTOR")
    print("="*60)
    print("Silakan isi profil Anda untuk mendapatkan rekomendasi proyek KPBU yang sesuai.")

    # Inisialisasi profil
    profile = {}

    # 1. Toleransi Risiko
    risk_options = {
        '1': 'Konservatif (Hanya proyek berisiko rendah)', 
        '2': 'Moderat (Proyek berisiko rendah-menengah)', 
        '3': 'Agresif (Semua tingkat risiko)'
    }
    while True:
        print("\n1. Apa toleransi risiko investasi Anda?")
        for key, value in risk_options.items():
            print(f"   {key}: {value}")
        choice = input("   Pilih nomor (1/2/3): ").strip()
        if choice in risk_options:
            if choice == '1':
                profile['toleransi_risiko'] = 'Konservatif'
            elif choice == '2':
                profile['toleransi_risiko'] = 'Moderat'
            else:
                profile['toleransi_risiko'] = 'Agresif'
            break
        else:
            print("   ❌ Pilihan tidak valid, silakan coba lagi.")

    # 2. Preferensi Sektor
    sektor_options = sorted([s for s in df['Sektor_Proyek'].unique() if pd.notna(s)])
    while True:
        print(f"\n2. Sektor apa yang Anda minati? (Tersedia {len(sektor_options)} sektor)")
        for i, sektor in enumerate(sektor_options, 1):
            # Hitung jumlah proyek per sektor
            count = len(df[df['Sektor_Proyek'] == sektor])
            print(f"   {i:2d}: {sektor} ({count} proyek)")
        
        print("\n   💡 Tip: Anda bisa memilih lebih dari satu sektor")
        print("   Masukkan nomor sektor, pisahkan dengan koma (contoh: 1,3,5)")
        choice_str = input("   Pilihan Anda: ").strip()
        
        try:
            chosen_indices = [int(i.strip()) - 1 for i in choice_str.split(',')]
            if all(0 <= idx < len(sektor_options) for idx in chosen_indices):
                profile['preferensi_sektor'] = [sektor_options[idx] for idx in chosen_indices]
                break
            else:
                print("   ❌ Satu atau lebih nomor pilihan tidak valid, silakan coba lagi.")
        except ValueError:
            print("   ❌ Format input salah. Harap masukkan nomor yang dipisahkan koma.")

    # 3. Horison Investasi
    horizon_options = {
        '1': 'Jangka Pendek (≤ 10 tahun)', 
        '2': 'Jangka Menengah (11-25 tahun)', 
        '3': 'Jangka Panjang (> 25 tahun)'
    }
    while True:
        print("\n3. Apa horison investasi yang Anda inginkan?")
        for key, value in horizon_options.items():
            print(f"   {key}: {value}")
        choice = input("   Pilih nomor (1/2/3): ").strip()
        if choice in horizon_options:
            if choice == '1':
                profile['horison_investasi'] = 'Jangka Pendek'
            elif choice == '2':
                profile['horison_investasi'] = 'Jangka Menengah'
            else:
                profile['horison_investasi'] = 'Jangka Panjang'
            break
        else:
            print("   ❌ Pilihan tidak valid, silakan coba lagi.")

    # 4. Preferensi Ukuran Investasi
    nilai_investasi = df['Nilai_Investasi_Total_IDR'].dropna()
    min_investasi = nilai_investasi.min()
    max_investasi = nilai_investasi.max()
    median_investasi = nilai_investasi.median()
    
    size_options = {
        '1': f'Kecil (< Rp {median_investasi/2/1e12:.1f} Triliun)', 
        '2': f'Menengah (Rp {median_investasi/2/1e12:.1f} - {median_investasi*2/1e12:.1f} Triliun)', 
        '3': f'Besar (> Rp {median_investasi*2/1e12:.1f} Triliun)',
        '4': 'Semua ukuran investasi'
    }
    
    while True:
        print("\n4. Preferensi ukuran investasi:")
        for key, value in size_options.items():
            print(f"   {key}: {value}")
        choice = input("   Pilih nomor (1/2/3/4): ").strip()
        if choice in size_options:
            if choice == '1':
                profile['ukuran_investasi'] = 'Kecil'
                profile['batas_investasi'] = median_investasi / 2
            elif choice == '2':
                profile['ukuran_investasi'] = 'Menengah'
                profile['batas_investasi'] = [median_investasi / 2, median_investasi * 2]
            elif choice == '3':
                profile['ukuran_investasi'] = 'Besar'
                profile['batas_investasi'] = median_investasi * 2
            else:
                profile['ukuran_investasi'] = 'Semua'
                profile['batas_investasi'] = None
            break
        else:
            print("   ❌ Pilihan tidak valid, silakan coba lagi.")

    return profile

# Panggil fungsi interaktif untuk mendapatkan profil investor
investor_profile = get_investor_profile_interactive(df_proyek)

print("\n" + "="*60)
print("           PROFIL INVESTOR TERSIMPAN")
print("="*60)
print(f"✅ Toleransi Risiko     : {investor_profile['toleransi_risiko']}")
print(f"✅ Preferensi Sektor    : {', '.join(investor_profile['preferensi_sektor'])}")
print(f"✅ Horison Investasi    : {investor_profile['horison_investasi']}")
print(f"✅ Ukuran Investasi     : {investor_profile['ukuran_investasi']}")
print("="*60)


           KUESIONER PROFIL INVESTOR
Silakan isi profil Anda untuk mendapatkan rekomendasi proyek KPBU yang sesuai.

1. Apa toleransi risiko investasi Anda?
   1: Konservatif (Hanya proyek berisiko rendah)
   2: Moderat (Proyek berisiko rendah-menengah)
   3: Agresif (Semua tingkat risiko)

2. Sektor apa yang Anda minati? (Tersedia 15 sektor)
    1: Air dan Sanitasi (13 proyek)
    2: Energi Terbarukan (5 proyek)
    3: Infrastruktur (1 proyek)
    4: Jalan dan Jembatan (19 proyek)
    5: Kesehatan (2 proyek)
    6: Konservasi Energi (2 proyek)
    7: Penelitian dan Pengembangan (2 proyek)
    8: Pengelolaan Sampah (2 proyek)
    9: Perumahan (1 proyek)
   10: Sumber Daya Air (1 proyek)
   11: Telekomunikasi dan Informatika (4 proyek)
   12: Transportasi (1 proyek)
   13: Transportasi Darat (2 proyek)
   14: Transportasi Laut (1 proyek)
   15: Transportasi Udara (1 proyek)

   💡 Tip: Anda bisa memilih lebih dari satu sektor
   Masukkan nomor sektor, pisahkan dengan koma (contoh: 1,3,5

In [3]:
# ---------------------------------------------------------------------------
# Langkah 2A: Profil Investor Demo (untuk pengujian)
# ---------------------------------------------------------------------------

# Profil investor demo untuk pengujian sistem
investor_profile_demo = {
    'toleransi_risiko': 'Moderat',  # Bisa menerima risiko rendah-menengah
    'preferensi_sektor': ['Jalan dan Jembatan', 'Air dan Sanitasi', 'Energi Terbarukan'],
    'horison_investasi': 'Jangka Menengah',  # 11-25 tahun
    'ukuran_investasi': 'Menengah',
    'batas_investasi': [2000000000000, 8000000000000]  # 2-8 Triliun IDR
}

print("=== PROFIL INVESTOR DEMO ===")
print(f"✅ Toleransi Risiko     : {investor_profile_demo['toleransi_risiko']}")
print(f"✅ Preferensi Sektor    : {', '.join(investor_profile_demo['preferensi_sektor'])}")
print(f"✅ Horison Investasi    : {investor_profile_demo['horison_investasi']}")
print(f"✅ Ukuran Investasi     : {investor_profile_demo['ukuran_investasi']}")
print(f"✅ Batas Investasi      : Rp {investor_profile_demo['batas_investasi'][0]/1e12:.1f} - {investor_profile_demo['batas_investasi'][1]/1e12:.1f} Triliun")
print("================================")

# Gunakan profil demo untuk testing
investor_profile = investor_profile_demo

=== PROFIL INVESTOR DEMO ===
✅ Toleransi Risiko     : Moderat
✅ Preferensi Sektor    : Jalan dan Jembatan, Air dan Sanitasi, Energi Terbarukan
✅ Horison Investasi    : Jangka Menengah
✅ Ukuran Investasi     : Menengah
✅ Batas Investasi      : Rp 2.0 - 8.0 Triliun


In [4]:
# ---------------------------------------------------------------------------
# Langkah 3: Implementasi Matching Engine
# ---------------------------------------------------------------------------

# --- Bagian A: Filter Keras (Rule-Based Filtering) ---
def filter_proyek_berdasarkan_risiko(df, profil_investor):
    """Fungsi untuk menyaring proyek berdasarkan toleransi risiko investor."""
    toleransi = profil_investor['toleransi_risiko']

    if toleransi == 'Konservatif':
        # Hanya proyek dengan risiko 'Rendah'
        allowed_risks = ['Rendah']
    elif toleransi == 'Moderat':
        # Proyek dengan risiko 'Rendah' dan 'Menengah'
        allowed_risks = ['Rendah', 'Menengah']
    elif toleransi == 'Agresif':
        # Semua proyek diizinkan
        return df
    else:
        # Default jika input tidak valid
        return pd.DataFrame()

    # Lakukan filter
    filtered_df = df[df['Profil_Risiko'].isin(allowed_risks)].copy()
    return filtered_df

def filter_proyek_berdasarkan_investasi(df, profil_investor):
    """Fungsi untuk menyaring proyek berdasarkan ukuran investasi yang diinginkan."""
    if profil_investor['ukuran_investasi'] == 'Semua':
        return df
    
    batas = profil_investor.get('batas_investasi')
    if batas is None:
        return df
        
    if profil_investor['ukuran_investasi'] == 'Kecil':
        # Investasi di bawah batas tertentu
        filtered_df = df[df['Nilai_Investasi_Total_IDR'] <= batas].copy()
    elif profil_investor['ukuran_investasi'] == 'Besar':
        # Investasi di atas batas tertentu
        filtered_df = df[df['Nilai_Investasi_Total_IDR'] >= batas].copy()
    elif profil_investor['ukuran_investasi'] == 'Menengah':
        # Investasi dalam rentang tertentu
        filtered_df = df[
            (df['Nilai_Investasi_Total_IDR'] >= batas[0]) & 
            (df['Nilai_Investasi_Total_IDR'] <= batas[1])
        ].copy()
    else:
        filtered_df = df.copy()
    
    return filtered_df

# --- Bagian B: Skoring Kecocokan (Content-Based Scoring) ---
def hitung_skor_kecocokan(proyek, profil_investor):
    """Fungsi untuk menghitung skor kecocokan sebuah proyek dengan profil investor."""

    # Definisikan bobot untuk setiap kriteria
    bobot = {
        'sektor': 0.4,
        'horison': 0.3,
        'risiko': 0.3
    }

    skor_akhir = 0

    # 1. Hitung Skor Kecocokan Sektor
    skor_sektor = 0
    if proyek['Sektor_Proyek'] in profil_investor['preferensi_sektor']:
        skor_sektor = 1.0
    else:
        # Berikan skor kecil untuk sektor yang tidak dipilih
        skor_sektor = 0.1

    # 2. Hitung Skor Kecocokan Horison Investasi
    skor_horison = 0
    durasi_proyek = proyek['Durasi_Konsesi_Tahun']
    horison_investor = profil_investor['horison_investasi']

    if horison_investor == 'Jangka Pendek' and durasi_proyek <= 10:
        skor_horison = 1.0
    elif horison_investor == 'Jangka Menengah' and 11 <= durasi_proyek <= 25:
        skor_horison = 1.0
    elif horison_investor == 'Jangka Panjang' and durasi_proyek > 25:
        skor_horison = 1.0
    else:
        # Beri skor parsial berdasarkan kedekatan
        if horison_investor == 'Jangka Pendek':
            skor_horison = max(0.2, 1 - abs(durasi_proyek - 5) * 0.05)
        elif horison_investor == 'Jangka Menengah':
            target = 18  # target tengah jangka menengah
            skor_horison = max(0.2, 1 - abs(durasi_proyek - target) * 0.03)
        else:  # Jangka Panjang
            skor_horison = max(0.2, 1 - abs(durasi_proyek - 30) * 0.02)

    # 3. Hitung Skor Kecocokan Risiko (bonus untuk risiko yang lebih sesuai)
    skor_risiko = 0.5  # skor default
    toleransi = profil_investor['toleransi_risiko']
    risiko_proyek = proyek['Profil_Risiko']
    
    if toleransi == 'Konservatif' and risiko_proyek == 'Rendah':
        skor_risiko = 1.0
    elif toleransi == 'Moderat' and risiko_proyek in ['Rendah', 'Menengah']:
        skor_risiko = 0.9 if risiko_proyek == 'Menengah' else 0.8
    elif toleransi == 'Agresif':
        skor_risiko = 1.0 if risiko_proyek == 'Tinggi' else 0.7

    # 4. Hitung Skor Total dengan Pembobotan
    skor_akhir = (bobot['sektor'] * skor_sektor) + (bobot['horison'] * skor_horison) + (bobot['risiko'] * skor_risiko)

    return skor_akhir

In [5]:
# ---------------------------------------------------------------------------
# Langkah 4: Proses Pencocokan dan Hasilkan Rekomendasi
# ---------------------------------------------------------------------------

print("\n--- Memulai Proses Pencocokan ---")

# 1. Terapkan Filter Keras - Risiko
proyek_filter_risiko = filter_proyek_berdasarkan_risiko(df_proyek, investor_profile)
print(f"Langkah 1: Ditemukan {len(proyek_filter_risiko)} proyek yang sesuai dengan toleransi risiko '{investor_profile['toleransi_risiko']}'.")

# 2. Terapkan Filter Keras - Ukuran Investasi
proyek_tersaring = filter_proyek_berdasarkan_investasi(proyek_filter_risiko, investor_profile)
print(f"Langkah 2: Ditemukan {len(proyek_tersaring)} proyek yang sesuai dengan ukuran investasi '{investor_profile['ukuran_investasi']}'.")

if not proyek_tersaring.empty:
    # 3. Hitung Skor Kecocokan untuk setiap proyek yang lolos filter
    proyek_tersaring['skor_kecocokan'] = proyek_tersaring.apply(
        lambda row: hitung_skor_kecocokan(row, investor_profile),
        axis=1
    )
    print("Langkah 3: Menghitung skor kecocokan untuk setiap proyek...")

    # 4. Urutkan proyek berdasarkan skor tertinggi
    rekomendasi_final = proyek_tersaring.sort_values(by='skor_kecocokan', ascending=False)

    # 5. Ambil Top 5 Rekomendasi
    top_5_rekomendasi = rekomendasi_final.head(5)

    print("Langkah 4: Mengurutkan proyek dan mengambil 5 rekomendasi teratas.")
    
    # Statistik tambahan
    print(f"\n--- Statistik Pencocokan ---")
    print(f"Skor kecocokan tertinggi: {rekomendasi_final['skor_kecocokan'].max():.2f}")
    print(f"Skor kecocokan rata-rata: {rekomendasi_final['skor_kecocokan'].mean():.2f}")
    print(f"Skor kecocokan terendah: {rekomendasi_final['skor_kecocokan'].min():.2f}")
else:
    top_5_rekomendasi = pd.DataFrame()
    print("❌ Tidak ada proyek yang memenuhi kriteria filtering.")



--- Memulai Proses Pencocokan ---
Langkah 1: Ditemukan 37 proyek yang sesuai dengan toleransi risiko 'Moderat'.
Langkah 2: Ditemukan 14 proyek yang sesuai dengan ukuran investasi 'Menengah'.
Langkah 3: Menghitung skor kecocokan untuk setiap proyek...
Langkah 4: Mengurutkan proyek dan mengambil 5 rekomendasi teratas.

--- Statistik Pencocokan ---
Skor kecocokan tertinggi: 0.97
Skor kecocokan rata-rata: 0.70
Skor kecocokan terendah: 0.45


In [6]:
# ---------------------------------------------------------------------------
# Langkah 5: Tampilkan Hasil Rekomendasi
# ---------------------------------------------------------------------------

print("\n" + "="*80)
print("                    HASIL REKOMENDASI PROYEK KPBU")
print("="*80)

if not top_5_rekomendasi.empty:
    for i, (idx, proyek) in enumerate(top_5_rekomendasi.iterrows(), 1):
        print(f"\n🏆 REKOMENDASI #{i}")
        print("-" * 50)
        print(f"📋 Nama Proyek      : {proyek['Nama_Proyek']}")
        print(f"🏭 Sektor           : {proyek['Sektor_Proyek']}")
        print(f"⚠️  Profil Risiko    : {proyek['Profil_Risiko']}")
        print(f"⏰ Durasi Konsesi   : {proyek['Durasi_Konsesi_Tahun']} tahun")
        print(f"💰 Nilai Investasi  : Rp {proyek['Nilai_Investasi_Total_IDR']/1e12:.2f} Triliun")
        print(f"⭐ Skor Kecocokan   : {proyek['skor_kecocokan']:.2%}")
        
        # Analisis mengapa proyek ini cocok
        print("📊 Analisis Kecocokan:")
        
        # Analisis sektor
        if proyek['Sektor_Proyek'] in investor_profile['preferensi_sektor']:
            print(f"   ✅ Sektor sesuai preferensi ({proyek['Sektor_Proyek']})")
        else:
            print(f"   ⚠️  Sektor tidak sesuai preferensi ({proyek['Sektor_Proyek']})")
        
        # Analisis horison
        durasi = proyek['Durasi_Konsesi_Tahun']
        horison = investor_profile['horison_investasi']
        if horison == 'Jangka Pendek' and durasi <= 10:
            print(f"   ✅ Horison investasi sesuai ({durasi} tahun)")
        elif horison == 'Jangka Menengah' and 11 <= durasi <= 25:
            print(f"   ✅ Horison investasi sesuai ({durasi} tahun)")
        elif horison == 'Jangka Panjang' and durasi > 25:
            print(f"   ✅ Horison investasi sesuai ({durasi} tahun)")
        else:
            print(f"   ⚠️  Horison investasi kurang sesuai ({durasi} tahun)")
        
        # Analisis risiko
        if investor_profile['toleransi_risiko'] == 'Konservatif' and proyek['Profil_Risiko'] == 'Rendah':
            print(f"   ✅ Risiko sesuai toleransi (Rendah)")
        elif investor_profile['toleransi_risiko'] == 'Moderat' and proyek['Profil_Risiko'] in ['Rendah', 'Menengah']:
            print(f"   ✅ Risiko sesuai toleransi ({proyek['Profil_Risiko']})")
        elif investor_profile['toleransi_risiko'] == 'Agresif':
            print(f"   ✅ Risiko sesuai toleransi ({proyek['Profil_Risiko']})")
        else:
            print(f"   ⚠️  Risiko mungkin di luar toleransi ({proyek['Profil_Risiko']})")

    # Ringkasan statistik
    print("\n" + "="*80)
    print("📈 RINGKASAN STATISTIK MATCHING")
    print("="*80)
    print(f"Total proyek yang dianalisis      : {len(df_proyek)}")
    print(f"Proyek yang lolos filter risiko   : {len(proyek_filter_risiko)}")
    print(f"Proyek yang lolos filter investasi: {len(proyek_tersaring)}")
    print(f"Rekomendasi yang ditampilkan      : {len(top_5_rekomendasi)}")
    print(f"Tingkat kecocokan tertinggi       : {top_5_rekomendasi['skor_kecocokan'].max():.2%}")
    
else:
    print("❌ TIDAK ADA PROYEK YANG COCOK")
    print("-" * 50)
    print("Mohon maaf, tidak ada proyek yang sesuai dengan profil Anda saat ini.")
    print("\n💡 Saran:")
    print("   • Coba perluas toleransi risiko Anda")
    print("   • Pertimbangkan sektor lain yang tersedia")
    print("   • Sesuaikan horison investasi atau ukuran investasi")

print("="*80)


                    HASIL REKOMENDASI PROYEK KPBU

🏆 REKOMENDASI #1
--------------------------------------------------
📋 Nama Proyek      : Fasilitas Pemanfaatan BMN pada Ibu Kota Lama
🏭 Sektor           : Energi Terbarukan
⚠️  Profil Risiko    : Menengah
⏰ Durasi Konsesi   : 22 tahun
💰 Nilai Investasi  : Rp 5.00 Triliun
⭐ Skor Kecocokan   : 97.00%
📊 Analisis Kecocokan:
   ✅ Sektor sesuai preferensi (Energi Terbarukan)
   ✅ Horison investasi sesuai (22 tahun)
   ✅ Risiko sesuai toleransi (Menengah)

🏆 REKOMENDASI #2
--------------------------------------------------
📋 Nama Proyek      : SPAM Regional Jatigede
🏭 Sektor           : Air dan Sanitasi
⚠️  Profil Risiko    : Menengah
⏰ Durasi Konsesi   : 28 tahun
💰 Nilai Investasi  : Rp 2.04 Triliun
⭐ Skor Kecocokan   : 88.00%
📊 Analisis Kecocokan:
   ✅ Sektor sesuai preferensi (Air dan Sanitasi)
   ⚠️  Horison investasi kurang sesuai (28 tahun)
   ✅ Risiko sesuai toleransi (Menengah)

🏆 REKOMENDASI #3
--------------------------------------

In [7]:
# ---------------------------------------------------------------------------
# Langkah 6: Tampilan Tabel Ringkasan untuk Perbandingan
# ---------------------------------------------------------------------------

if not top_5_rekomendasi.empty:
    print("\n" + "="*100)
    print("                           TABEL RINGKASAN REKOMENDASI")
    print("="*100)
    
    # Buat tabel ringkasan
    summary_table = top_5_rekomendasi.copy()
    summary_table['Ranking'] = range(1, len(summary_table) + 1)
    summary_table['Nilai_Investasi_Triliun'] = summary_table['Nilai_Investasi_Total_IDR'] / 1e12
    summary_table['Skor_Persen'] = summary_table['skor_kecocokan'] * 100
    
    # Pilih kolom untuk ditampilkan
    display_columns = [
        'Ranking', 'Nama_Proyek', 'Sektor_Proyek', 'Profil_Risiko', 
        'Durasi_Konsesi_Tahun', 'Nilai_Investasi_Triliun', 'Skor_Persen'
    ]
    
    # Format tabel
    summary_display = summary_table[display_columns].copy()
    summary_display['Nilai_Investasi_Triliun'] = summary_display['Nilai_Investasi_Triliun'].map('{:.2f}'.format)
    summary_display['Skor_Persen'] = summary_display['Skor_Persen'].map('{:.1f}%'.format)
    
    # Rename kolom untuk display
    summary_display.columns = [
        'Rank', 'Nama Proyek', 'Sektor', 'Risiko', 
        'Durasi (Thn)', 'Investasi (T)', 'Match Score'
    ]
    
    print(summary_display.to_string(index=False))
    print("="*100)
    
    # Export ke CSV jika diinginkan
    export_path = 'data/rekomendasi_investor.csv'
    try:
        summary_table[display_columns].to_csv(export_path, index=False)
        print(f"\n💾 Hasil rekomendasi telah disimpan ke: {export_path}")
    except Exception as e:
        print(f"\n⚠️  Gagal menyimpan file: {e}")
        
else:
    print("\n❌ Tidak ada data untuk ditampilkan dalam tabel ringkasan.")

print(f"\n🎯 Sistem matching investor-proyek KPBU telah berhasil dijalankan!")
print(f"📊 Untuk analisis lebih lanjut, Anda dapat menggunakan data yang telah difilter.")


                           TABEL RINGKASAN REKOMENDASI
 Rank                                      Nama Proyek             Sektor   Risiko  Durasi (Thn) Investasi (T) Match Score
    1     Fasilitas Pemanfaatan BMN pada Ibu Kota Lama  Energi Terbarukan Menengah            22          5.00       97.0%
    2                           SPAM Regional Jatigede   Air dan Sanitasi Menengah            28          2.04       88.0%
    3                     SPAM Regional Karian-Serpong   Air dan Sanitasi Menengah            29          2.43       87.1%
    4 Proyek KPBU Bendungan Merangin Terintegrasi SPAM   Air dan Sanitasi Menengah            29          4.77       87.1%
    5       Penggantian dan/atau Duplikasi Jembatan CH Jalan dan Jembatan Menengah            30          2.20       86.2%

💾 Hasil rekomendasi telah disimpan ke: data/rekomendasi_investor.csv

🎯 Sistem matching investor-proyek KPBU telah berhasil dijalankan!
📊 Untuk analisis lebih lanjut, Anda dapat menggunakan data yang telah